# Masked gaussian smoothing of dense and sparse signals

### <font color='red'> After clicking on a cell, press "Shift+Enter" to run the code, or click on the "Run" button in the toolbar above.<br>

### Replace "..." signs with the appropriate path to your data.
</font>

In [ ]:
from tapenade.preprocessing import masked_gaussian_smoothing
from tapenade import get_path_to_demo_folder
from pathlib import Path
import tifffile
import matplotlib.pyplot as plt
from skimage.measure import regionprops
import numpy as np
try:
    import napari
    napari_available = True
except ImportError:
    print("napari is not installed, some optional parts of the notebook will not be accessible")
    napari_available = False

This notebook presents the masked gaussian smoothing method in the case of 
1. dense image smoothing 
2. object instances smoothing to compute densities 
3. dense smoothing of sparse data (defined locally at specific positions)  

## 1. Smoothing images

### 1.1 Without a mask

In [ ]:
path_to_data = get_path_to_demo_folder()
image = tifffile.imread(path_to_data / 'bra_isotropized.tif')
mid_plane_ind = int(image.shape[0] // 2)

Define a smoothing sigma that will be used as the standard deviation of the gaussian kernel. Then, smooth the image. 

In [ ]:
sigma = 10

smoothed_image = masked_gaussian_smoothing(image, sigmas=sigma)

Show the midplane:

In [ ]:
fig, axes = plt.subplots(1, 2)
axes[0].imshow(image[mid_plane_ind], cmap='gray')
axes[1].imshow(smoothed_image[mid_plane_ind], cmap='gray')

You can see that the smoothed signal extends beyond the contours of the original signal, because no mask has been used. This also introduces a bias in the smoothed signal <u>inside</u> the sample, because values outside the sample will be considered in the smoothing process.

### 1.2 With a mask

Load a mask of the inside of the sample. It can for example be obtained via our preprocessing `compute_mask` function. 

In [ ]:
mask = tifffile.imread(path_to_data / 'mask_def_corr.tif')

Smooth signal with the mask:

In [ ]:
smoothed_image_masked = masked_gaussian_smoothing(image, mask=mask, sigmas=sigma)

In [ ]:
fig, axes = plt.subplots(1,2)
axes[0].imshow(smoothed_image_masked[mid_plane_ind], cmap='gray')
diff = axes[1].imshow(smoothed_image[mid_plane_ind] - smoothed_image_masked[mid_plane_ind], cmap='RdBu')
fig.colorbar(diff, ax=axes[1], orientation='horizontal')
axes[1].set_title("Difference between smoothed and masked smoothed image", fontsize=8)

Now, the signal stops outside the sample, and the bias inside the sample is reduced (it manifests as increased signal close to the sample boundaries).

If we are interested in a signal that expresses in specific regions of the sample, e.g a gene that fluoresces only in nuclei, there remain a last bias that comes from the fact the the space <u> between</u> the nuclei should not be considered in the smoothing process. This bias can be removed by specifying a mask specifically for these regions.

### 1.3 With a mask of object instances

Load labels for object instances, e.g nuclei. It can be obtained via our `segmentation.predict_stardist` function. The mask that restricts the smoothing to the nuclei is obtained by directly converting labels into a binary mask.

In [ ]:
labels = tifffile.imread(path_to_data / 'labels_def_corr.tif')

mask_nuclei_instances = labels.astype(bool)

<font color='red'>Note that we only require a mask of the subregion that needs to be smoothed, not actually the segmented instances. If you want to smooth a signal that cannot be segmented into individual instances, you can simply obtain the mask (e.g for membrane signal).</font>

Now smooth the signal with the mask of the nuclei (to restrict the smoothing to the nuclei), and the mask of the inside of the sample (to restrict the smoothing result to the sample boundaries):

In [ ]:
smoothed_image_masked2 = masked_gaussian_smoothing(image, mask=labels, sigmas=sigma)

In [ ]:
# plt.imshow(smoothed_image_masked2[mid_plane_ind], cmap='gray')
fig, axes = plt.subplots(1,2)
axes[0].imshow(smoothed_image_masked2[mid_plane_ind], cmap='gray')
foo = smoothed_image_masked[mid_plane_ind]
foo[~mask_nuclei_instances[mid_plane_ind]] = 0
diff = axes[1].imshow(foo - smoothed_image_masked2[mid_plane_ind], cmap='RdBu', vmin=-300, vmax=300)
fig.colorbar(diff, ax=axes[1], orientation='horizontal')
axes[1].set_title("Difference between masked smoothed and\n masked with labels smooth images", fontsize=8)

This corrects the second bias, the smoothed signal is defined everywhere in the signal, but only the signal inside the nuclei is considered in the smoothing process.

In [ ]:
if napari_available:
    viewer = napari.Viewer()

    viewer.add_image(image, name='image')
    viewer.add_image(smoothed_image, name='smoothed_image')
    viewer.add_image(mask, name='mask')
    viewer.add_image(smoothed_image_masked, name='smoothed_image_masked')
    viewer.add_labels(labels, name='labels')
    viewer.add_image(smoothed_image_masked2, name='smoothed_image_masked2')

    viewer.grid.enabled = True
    viewer.reset_view()

## 2. Smoothing segmented object instances (labels) to get objects density or volume fraction

To compute density-related fields (object density and volume fraction), we need to smooth the labels of the object instances.

Smart by loading a mask of the inside of the sample, and the labels of the object instances.

### Object density

To compute object density (number of objects per unit volume), we define a non-smoothed array of ones with the same shape as the labels, and smooth it with the mask of the inside of the sample. This will count the number of 1's inside the kernel, weighted by the kernel values.

In [ ]:
props = regionprops(labels)
object_centroid_inds = tuple(np.array([prop.centroid for prop in props]).T.astype(int))

signal_object_centers = np.zeros_like(labels, dtype=bool)
signal_object_centers[object_centroid_inds] = True

Smooth the array of ones, with the mask of the inside of the sample to restrict the smoothing to the sample boundaries:

In [ ]:
object_density_field = masked_gaussian_smoothing(
    signal_object_centers, 
    mask=mask,
    sigmas=sigma
)

In [ ]:
im=plt.imshow(object_density_field[mid_plane_ind], cmap='inferno')
plt.title("Object density field smoothed (masked correction)", fontsize=8)
fig.colorbar(im)

### Volume fraction

For the volume fraction (fraction of the volume occupied by the objects), the initial non-smoothed signal corresponds to a binary mask of the object instances. This equivalent to saying that at the pixel scale, the volume fraction is 100% inside the objects and 0% outside.

In [ ]:
mask_nuclei_instances = labels.astype(bool)

Smooth the signal, with the mask of the inside of the sample to restrict the smoothing to the sample boundaries:

In [ ]:
volume_fraction_field = masked_gaussian_smoothing(
    mask_nuclei_instances, 
    mask=mask,
    sigmas=sigma
)

In [ ]:
im=plt.imshow(volume_fraction_field[mid_plane_ind], cmap='viridis')
plt.title("Volume fraction field smoothed (masked correction)", fontsize=8)
fig.colorbar(im)

In [ ]:
if napari_available:
    viewer = napari.Viewer()

    viewer.add_labels(labels, name='labels')
    viewer.add_image(mask, name='mask')
    viewer.add_image(object_density_field, name='object_density_field', colormap='inferno')
    viewer.add_image(volume_fraction_field, name='volume_fraction_field', colormap='viridis')

    viewer.grid.enabled = True
    viewer.reset_view()

## 3. Smoothing object-located signals

In the case of sparse, i.e signals that are defined only at specific positions, we can use the same method to smooth the signal, if the result is expected to be defined everywhere in the sample.

In this case, we provide a mask that restricts the computation at the positions at which the sparse signal is defined.

In this case, we will illustrate the method by computing a continuous field of object volume (e.g nuclei) from the segmented instances.

The initial non-smoothed signal is an array that is non-zero only at the centroid of the objects, and whose value at these positions is the volume of the object.

In [ ]:
props = regionprops(labels)
object_centroid_inds = tuple(np.array([prop.centroid for prop in props]).T.astype(int))
object_volumes = np.array([prop.area for prop in props]) # for 3D images, prop.area is the volume

# create a mask with object centers
mask_object_centers = np.zeros_like(labels, dtype=bool)
mask_object_centers[object_centroid_inds] = True

# create the initial non-smoothed signal of object volumes
signal_object_volumes = np.zeros_like(labels, dtype=float)
signal_object_volumes[object_centroid_inds] = object_volumes

Smooth the signal, with the mask of the inside of the sample to restrict the smoothing to the sample boundaries, and the mask of the object instances to restrict the smoothing to the positions at which the sparse signal is defined:

In [ ]:
smoothed_volume_field = masked_gaussian_smoothing(
    signal_object_volumes, 
    mask=mask,
    mask_for_volume=mask_object_centers,
    sigmas=sigma
)

In [ ]:
im=plt.imshow(smoothed_volume_field[mid_plane_ind], cmap='cividis')
plt.title("Smoothed volume field (masked correction)", fontsize=8)
fig.colorbar(im)

In [ ]:
if napari_available:
    viewer = napari.Viewer()

    viewer.add_labels(labels, name='labels')
    viewer.add_image(mask, name='mask')
    viewer.add_image(smoothed_volume_field, name='smoothed_volume_field', colormap='cividis')

    viewer.grid.enabled = True
    viewer.reset_view()